In [43]:
import pandas as pd
import re
import os
from google.cloud import storage
import numpy as np

# Nếu bạn có service account key file, uncomment dòng dưới:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/service-account-key.json'

df = pd.read_csv("batdongsan.csv")


In [44]:
df.head(5)


,Diện tích sử dụng,Mã BĐS,price,Pháp lý,Ngày đăng,Nhà tắm,Phòng ngủ,address,Diện tích đất
0,100 m2,22712100,5 tỷ 800 triệu,Sổ đỏ,15/10/2025,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN
1,80 m2,22712094,4 tỷ 350 triệu,Sổ đỏ,15/10/2025,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
2,90 m2,22711721,4 tỷ 580 triệu,Sổ đỏ,15/10/2025,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN
3,120 m2,22711722,4 tỷ 700 triệu,Sổ đỏ,15/10/2025,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
4,100 m2,22711723,5 tỷ 400 triệu,Sổ đỏ,15/10/2025,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN


In [45]:
df = df.rename(columns={
    'Diện tích sử dụng' : 'dien_tich_su_dung_m2',
    'Mã BĐS' : 'ma_bds',
    'price' : 'gia_ty',
    'Pháp lý' : 'phap_ly',
    'Ngày đăng' : 'ngay_dang',
    'Nhà tắm'  : 'nha_tam',
    'Phòng ngủ': 'phong_ngu',
    'address': 'dia_chi',
    'Diện tích đất': 'dien_tich_dat_m2'
})

In [46]:
df.head(5)

,dien_tich_su_dung_m2,ma_bds,gia_ty,phap_ly,ngay_dang,nha_tam,phong_ngu,dia_chi,dien_tich_dat_m2
0,100 m2,22712100,5 tỷ 800 triệu,Sổ đỏ,15/10/2025,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN
1,80 m2,22712094,4 tỷ 350 triệu,Sổ đỏ,15/10/2025,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
2,90 m2,22711721,4 tỷ 580 triệu,Sổ đỏ,15/10/2025,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN
3,120 m2,22711722,4 tỷ 700 triệu,Sổ đỏ,15/10/2025,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
4,100 m2,22711723,5 tỷ 400 triệu,Sổ đỏ,15/10/2025,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN


In [47]:
# dien  tich su dung
df['dien_tich_su_dung_m2'] = df['dien_tich_su_dung_m2'].str.extract(r'(\d+(?:\.\d+)?)').astype(float)



In [ ]:


# Làm sạch dữ liệu - chuyển string thành NaN
def clean_room_data(value):
    if pd.isna(value):
        return None
    if isinstance(value, str):
        # Nếu là string như 'Nhà tắm', 'Phòng ngủ' thì chuyển thành NaN
        if value.lower() in ['nhà tắm', 'phòng ngủ', 'phong ngu', 'nha tam']:
            return None
        # Nếu là số dạng string, chuyển thành số
        try:
            return float(value)
        except:
            return None
    return value

# Làm sạch dữ liệu
df['nha_tam'] = df['nha_tam'].apply(clean_room_data)
df['phong_ngu'] = df['phong_ngu'].apply(clean_room_data)


# Phân tích mối quan hệ giữa diện tích và số phòng
# Tính trung bình số phòng theo diện tích
df_with_rooms = df[(df['nha_tam'].notna()) & (df['phong_ngu'].notna())].copy()
if len(df_with_rooms) > 0:
    # Tạo bins cho diện tích
    df_with_rooms['dien_tich_bin'] = pd.cut(df_with_rooms['dien_tich_su_dung_m2'], 
                                           bins=[0, 50, 80, 120, 200, float('inf')], 
                                           labels=['<50m²', '50-80m²', '80-120m²', '120-200m²', '>200m²'])
    
    # Tính trung bình số phòng theo từng bin diện tích
    avg_rooms_by_area = df_with_rooms.groupby('dien_tich_bin', observed=True).agg({
        'nha_tam': 'mean',
        'phong_ngu': 'mean'
    }).round(0)
    
    
    # Hàm fill dựa trên diện tích
    def fill_rooms_by_area(row):
        if pd.isna(row['nha_tam']) or pd.isna(row['phong_ngu']):
            area = row['dien_tich_su_dung_m2']
            if pd.isna(area):
                return row['nha_tam'], row['phong_ngu']
            
            # Xác định bin diện tích
            if area < 50:
                bin_name = '<50m²'
            elif area < 80:
                bin_name = '50-80m²'
            elif area < 120:
                bin_name = '80-120m²'
            elif area < 200:
                bin_name = '120-200m²'
            else:
                bin_name = '>200m²'
            
            # Fill từ trung bình của bin tương ứng
            if bin_name in avg_rooms_by_area.index:
                nha_tam_fill = avg_rooms_by_area.loc[bin_name, 'nha_tam'] if pd.isna(row['nha_tam']) else row['nha_tam']
                phong_ngu_fill = avg_rooms_by_area.loc[bin_name, 'phong_ngu'] if pd.isna(row['phong_ngu']) else row['phong_ngu']
                return nha_tam_fill, phong_ngu_fill
        
        return row['nha_tam'], row['phong_ngu']
    
    # Apply hàm fill
    df[['nha_tam', 'phong_ngu']] = df.apply(fill_rooms_by_area, axis=1, result_type='expand')
    
    # Fill các giá trị còn lại bằng trung bình chung
    if df['nha_tam'].isna().sum() > 0:
        df['nha_tam'].fillna(df['nha_tam'].mean(), inplace=True)
    if df['phong_ngu'].isna().sum() > 0:
        df['phong_ngu'].fillna(df['phong_ngu'].mean(), inplace=True)
    
    # Convert về int - làm tròn trước khi convert
    df['nha_tam'] = df['nha_tam'].round().astype('Int64')
    df['phong_ngu'] = df['phong_ngu'].round().astype('Int64')
    

=== TRƯỚC KHI FILL NaN ===
Số lượng NaN trong nha_tam: 0
Số lượng NaN trong phong_ngu: 0

=== KIỂM TRA GIÁ TRỊ STRING ===
Các giá trị unique trong nha_tam:
nha_tam
4.000000     1755
3.000000     1114
5.000000      629
2.000000      582
6.000000      579
1.000000      377
8.000000      205
5.073238      153
7.000000      114
10.000000      84
Name: count, dtype: int64

Các giá trị unique trong phong_ngu:
phong_ngu
4.000000    1612
3.000000    1121
5.000000     827
6.000000     656
2.000000     596
1.000000     223
8.000000     220
5.205747     150
7.000000     110
9.000000      76
Name: count, dtype: int64

Sau khi làm sạch - NaN trong nha_tam: 0
Sau khi làm sạch - NaN trong phong_ngu: 0

=== PHÂN TÍCH MỐI QUAN HỆ ===
Trung bình số phòng theo diện tích:
               nha_tam  phong_ngu
dien_tich_bin                    
<50m²              4.0        4.0
50-80m²            4.0        5.0
80-120m²           6.0        6.0
120-200m²          6.0        6.0
>200m²             8.0        8.0

In [40]:
df = df.drop_duplicates(subset=['ma_bds']).reset_index(drop=True)
df['ma_bds'] = df['ma_bds'].astype(str)
df['nha_tam'] = pd.to_numeric(df['nha_tam'], errors='coerce').astype('Int64')
df['phong_ngu'] = pd.to_numeric(df['phong_ngu'], errors='coerce').astype('Int64')

In [41]:
def chuan_hoa_gia(x):
    x = str(x).lower()
    ty = 0
    trieu = 0

    # Lấy phần 'tỷ'
    match_ty = re.search(r'(\d+(?:[\.,]\d+)?)\s*t[ỷi]', x)
    if match_ty:
        ty = float(match_ty.group(1).replace(',', '.'))

    # Lấy phần 'triệu'
    match_trieu = re.search(r'(\d+(?:[\.,]\d+)?)\s*triệu', x)
    if match_trieu:
        trieu = float(match_trieu.group(1).replace(',', '.'))

    # Đổi về tỷ đồng
    return round(ty + trieu/1000, 3)

def tach_dia_chi(dia_chi):
    dia_chi = str(dia_chi)
    
    # Tên đường (trước dấu phẩy đầu tiên)
    ten_duong = dia_chi.split(",")[0].strip()

    # Tên phường/xã
    phuong = re.search(r'Phường\s+([^,]+)', dia_chi)
    phuong = phuong.group(1).strip() if phuong else None

    # Tên quận/huyện
    quan = re.search(r'Quận\s+([^,]+)', dia_chi)
    quan = quan.group(1).strip() if quan else None

    # Thành phố / Tỉnh
    tp = re.search(r'(Thành phố|TP\.?|Tỉnh)\s+([^,]+)', dia_chi)
    tp = tp.group(2).strip() if tp else "Hà Nội"

    # Gộp tên đường + phường
    phuong_xa = f"{ten_duong} - Phường {phuong}" if phuong else ten_duong

    return pd.Series([phuong_xa, quan, tp])


# Hàm làm sạch diện tích
def clean_dien_tich(value):
    if pd.isna(value):
        return None
    value = str(value).lower().strip()
    
    # Tìm số đầu tiên trong chuỗi (có thể có thập phân)
    match = re.search(r'(\d+(\.\d+)?)', value)
    if match:
        return float(match.group(1))
    else:
        return None



df['dien_tich_dat_m2'] = df['dien_tich_dat_m2'].apply(clean_dien_tich)
df['gia(ty)'] = df['gia_ty'].apply(chuan_hoa_gia)
df[['phuong_xa', 'quan_huyen', 'thanh_pho']] = df['dia_chi'].apply(tach_dia_chi)




In [10]:
for c in ['phap_ly','phuong_xa','quan_huyen','thanh_pho']:
    df[c] = df[c].astype(str).str.strip()

In [11]:
df['ngay_dang'] = pd.to_datetime(df['ngay_dang'], format='%d/%m/%Y', errors='coerce')

In [12]:
df.head(5)

,dien_tich_su_dung_m2,ma_bds,gia_ty,phap_ly,ngay_dang,nha_tam,phong_ngu,dia_chi,dien_tich_dat_m2,gia(ty),phuong_xa,quan_huyen,thanh_pho
0,100.0,22712100,5 tỷ 800 triệu,Sổ đỏ,2025-10-15,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN,5.80,Đại La - Phường Đồng Nhân,Hai Bà Trưng,Hà Nội
1,80.0,22712094,4 tỷ 350 triệu,Sổ đỏ,2025-10-15,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN,4.35,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội
2,90.0,22711721,4 tỷ 580 triệu,Sổ đỏ,2025-10-15,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN,4.58,Nhân Hòa - Phường Nhân Chính,Thanh Xuân,Hà Nội
3,120.0,22711722,4 tỷ 700 triệu,Sổ đỏ,2025-10-15,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN,4.70,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội
4,100.0,22711723,5 tỷ 400 triệu,Sổ đỏ,2025-10-15,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN,5.40,Yên Lạc - Phường Vĩnh Tuy,Hai Bà Trưng,Hà Nội


In [13]:

df_tmp = df[['ma_bds','dien_tich_su_dung_m2','gia(ty)','phap_ly','ngay_dang','phuong_xa','quan_huyen','thanh_pho','nha_tam','phong_ngu','dien_tich_dat_m2']]


df_tmp = df_tmp[(df_tmp['gia(ty)'] != 0.0) & (df_tmp['gia(ty)'].notna())]


In [14]:
# KIỂM TRA BUSINESS LOGIC VALIDATION
print("=== BUSINESS LOGIC VALIDATION ===")

# 1. Kiểm tra giá hợp lý
print("1. GIÁ HỢP LÝ:")
print(f"   - Giá min: {df_tmp['gia(ty)'].min():.3f} tỷ")
print(f"   - Giá max: {df_tmp['gia(ty)'].max():.3f} tỷ")
print(f"   - Giá trung bình: {df_tmp['gia(ty)'].mean():.3f} tỷ")

# 2. Kiểm tra diện tích hợp lý
print("\n2. DIỆN TÍCH HỢP LÝ:")
print(f"   - Diện tích min: {df_tmp['dien_tich_su_dung_m2'].min():.1f} m²")
print(f"   - Diện tích max: {df_tmp['dien_tich_su_dung_m2'].max():.1f} m²")
print(f"   - Diện tích trung bình: {df_tmp['dien_tich_su_dung_m2'].mean():.1f} m²")

# 3. Tính giá/m²
df_tmp['gia_per_m2'] = (df_tmp['gia(ty)'] * 1000) / df_tmp['dien_tich_su_dung_m2']
print(f"\n3. GIÁ/M²:")
print(f"   - Giá/m² min: {df_tmp['gia_per_m2'].min():.0f} triệu VND/m²")
print(f"   - Giá/m² max: {df_tmp['gia_per_m2'].max():.0f} triệu VND/m²")
print(f"   - Giá/m² trung bình: {df_tmp['gia_per_m2'].mean():.0f} triệu VND/m²")


=== BUSINESS LOGIC VALIDATION ===
1. GIÁ HỢP LÝ:
   - Giá min: 0.002 tỷ
   - Giá max: 86900.000 tỷ
   - Giá trung bình: 51.169 tỷ

2. DIỆN TÍCH HỢP LÝ:
   - Diện tích min: 1.0 m²
   - Diện tích max: 6300.0 m²
   - Diện tích trung bình: 126.7 m²

3. GIÁ/M²:
   - Giá/m² min: 0 triệu VND/m²
   - Giá/m² max: 790000 triệu VND/m²
   - Giá/m² trung bình: 714 triệu VND/m²


In [15]:
df_tmp.to_csv("batdongsan_clean.csv", index=False, encoding="utf-8-sig")


In [16]:
df_tmp[df_tmp['gia(ty)'] == 0.0]

,ma_bds,dien_tich_su_dung_m2,gia(ty),phap_ly,ngay_dang,phuong_xa,quan_huyen,thanh_pho,nha_tam,phong_ngu,dien_tich_dat_m2,gia_per_m2


In [19]:

# Thay thế NaN trong dien_tich_su_dung_m2 bằng dien_tich_dat_m2
mask_su_dung_na = df_tmp['dien_tich_su_dung_m2'].isna() & df_tmp['dien_tich_dat_m2'].notna()
df_tmp.loc[mask_su_dung_na, 'dien_tich_su_dung_m2'] = df_tmp.loc[mask_su_dung_na, 'dien_tich_dat_m2']

# Thay thế NaN trong dien_tich_dat_m2 bằng dien_tich_su_dung_m2
mask_dat_na = df_tmp['dien_tich_dat_m2'].isna() & df_tmp['dien_tich_su_dung_m2'].notna()
df_tmp.loc[mask_dat_na, 'dien_tich_dat_m2'] = df_tmp.loc[mask_dat_na, 'dien_tich_su_dung_m2']



In [51]:
df_tmp.head(5)


,ma_bds,dien_tich_su_dung_m2,gia(ty),phap_ly,ngay_dang,phuong_xa,quan_huyen,thanh_pho,nha_tam,phong_ngu,dien_tich_dat_m2,gia_per_m2
0,22712100,100.0,5.80,Sổ đỏ,2025-10-15,Đại La - Phường Đồng Nhân,Hai Bà Trưng,Hà Nội,1,2,100.0,58.000000
1,22712094,80.0,4.35,Sổ đỏ,2025-10-15,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội,2,2,80.0,54.375000
2,22711721,90.0,4.58,Sổ đỏ,2025-10-15,Nhân Hòa - Phường Nhân Chính,Thanh Xuân,Hà Nội,2,3,90.0,50.888889
3,22711722,120.0,4.70,Sổ đỏ,2025-10-15,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội,2,3,120.0,39.166667
4,22711723,100.0,5.40,Sổ đỏ,2025-10-15,Yên Lạc - Phường Vĩnh Tuy,Hai Bà Trưng,Hà Nội,2,3,100.0,54.000000
